In [1]:
import numpy as np
import nltk
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
nltk.download('punkt')
nltk.download('webtext')
nltk.download('nps_chat')
nltk.download('brown')
nltk.download('reuters')

[nltk_data] Downloading package punkt to /home/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package webtext to /home/marco/nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package nps_chat to /home/marco/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package brown to /home/marco/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to /home/marco/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [3]:
data = []
for fileid in nltk.corpus.webtext.fileids():
    data.extend(nltk.corpus.webtext.sents(fileid))
for fileid in nltk.corpus.nps_chat.fileids():
    data.extend(nltk.corpus.nps_chat.posts(fileid))
for fileid in nltk.corpus.brown.fileids():
    data.extend(nltk.corpus.brown.sents(fileid))
for fileid in nltk.corpus.reuters.fileids():
    data.extend(nltk.corpus.reuters.sents(fileid))
for sent in data:
    for i in range(len(sent)):
        sent[i] = sent[i].lower()
words = set()
for sent in data:
    words.update(sent)
word_index_max = len(words)
word_index = {w:i+1 for i,w in enumerate(words)}
word_lookup = {i+1:w for i,w in enumerate(words)}
for sent in data:
    for i in range(len(sent)):
        sent[i] = word_index[sent[i]]

In [4]:
best_embedding_dim = 50 #int(round(len(word_index)**(1/4)))
best_embedding_dim

50

In [5]:
sampling_table = keras.preprocessing.sequence.make_sampling_table(word_index_max+1)

In [6]:
def generate_batch():
    while True:
        i = np.random.randint(0, len(data))
        ssamples, slabels = keras.preprocessing.sequence.skipgrams(
            data[i], word_index_max+1, sampling_table=sampling_table, window_size=4)
        if not ssamples:
            continue
        ssamples = np.array(ssamples)
        yield [ssamples[:,0],ssamples[:,1]], np.array(slabels, dtype=np.float)

In [7]:
X = X_input = keras.layers.Input((1,), dtype='int32')
X = X_embedding = keras.layers.Embedding(word_index_max+1, best_embedding_dim)(X)
X = keras.layers.Flatten()(X)
M_embedding = keras.Model(X_input, X)
X_input_a = X_a = keras.layers.Input((1,), dtype='int32')
X_input_b = X_b = keras.layers.Input((1,), dtype='int32')
X_a = M_embedding(X_a)
X_b = M_embedding(X_b)
X = keras.layers.Dot(-1, normalize=True)([X_a,X_b])
X = keras.layers.Dense(1, activation='sigmoid',
                       kernel_constraint=keras.constraints.NonNeg())(X)
M = keras.Model([X_input_a,X_input_b], X)
M.compile('rmsprop', 'binary_crossentropy', ['acc', 'mae'])
M.summary()

Instructions for updating:
dim is deprecated, use axis instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 50)           3734050     input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot)                     (None, 1)     

In [8]:
M.fit_generator(generate_batch(), steps_per_epoch=len(data))

Epoch 1/1
148351/148351 [==============================] - 8348s 56ms/step - loss: 0.5659 - acc: 0.7286 - mean_absolute_error: 0.3785


In [9]:
M.save('./models/word-embedding-analogic/model.hdf5')

In [10]:
M = keras.models.load_model('./models/word-embedding-analogic/model.hdf5')

In [11]:
embedding = M.get_layer(index=2).get_layer(index=1).get_weights()[0]

In [36]:
def test(a, b, c, use_cos=False):
    v_a = embedding[word_index[a]]
    v_b = embedding[word_index[b]]
    v_c = embedding[word_index[c]]
    v_d = v_c + (v_b - v_a)
    if use_cos:
        cos_sim = np.dot(embedding,
            v_d / np.linalg.norm(v_d, axis=-1)) / np.linalg.norm(embedding, axis=-1)
        i_d = np.argsort(cos_sim)
        return [word_lookup.get(i_d_) for i_d_ in reversed(i_d[-25:])]
    else:
        norm = np.linalg.norm(embedding - v_d, axis=-1)
        i_d = np.argsort(norm)
        return [word_lookup.get(i_d_) for i_d_ in i_d[:25]]

In [38]:
with open('./utils/questions-words.txt', 'r') as f:
    for line in f:
        if not line or line[0] == ':':
            continue
        a, b, c, d = line.strip().lower().split(' ')
        if not all((w in word_index) for w in (a,b,c,d)):
            continue
        pred = test(a, b, c, use_cos=True) + test(a, b, c, use_cos=False)
        if d in pred:
            print(a, b, c, d)

rome italy paris france
anchorage alaska philadelphia pennsylvania
boy girl husband wife
brothers sisters he she
brothers sisters his her
dad mom his her
dad mom husband wife
father mother his her
father mother husband wife
grandfather grandmother his her
grandfather grandmother boy girl
grandfather grandmother father mother
grandpa grandma his her
grandpa grandma father mother
grandson granddaughter his her
grandson granddaughter boy girl
he she his her
his her he she
husband wife boy girl
king queen his her
nephew niece boy girl
nephew niece father mother
nephew niece his her
nephew niece husband wife
policeman policewoman boy girl
policeman policewoman father mother
policeman policewoman his her
prince princess boy girl
prince princess father mother
prince princess his her
son daughter his her
sons daughters boy girl
sons daughters father mother
sons daughters his her
uncle aunt man woman
most mostly lucky luckily
possible possibly lucky luckily
cheap cheaper good better
good better